<a href="https://colab.research.google.com/github/Subamprasad/Zenml/blob/main/Zenml1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ML Pipelines with **Zenml**


In [1]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2

import IPython

IPython.Application.instance().kernel.do_shutdown(restart=True)

⠼ Installing integrations...


{'status': 'ok', 'restart': True}

In [1]:
NGROK_TOKEN =  "2y2XZaE9ID4nWrzKyY4SNiBfYqE_7R83fKaA38LoEfdsWxMnF"

In [2]:
from zenml.environment import Environment

if Environment.in_google_colab():
    !pip install pyngrok
    !ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!rm -rf .zen
!zenml init

⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initiali

Example Experimentation ML coad

In [4]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  """Train and test a Scikit-Learn SVC classifier on digits"""
  digits = load_digits()
  X_train, X_test, y_train, y_test = train_test_split(
      digits.data, digits.target, random_state=0
  )
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  test_acc = model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")

train_test()

Test accuracy: 0.9955555555555555


pipelines 1. importer   2. svc trainer     3. evaluator


In [6]:
from zenml import step
from typing_extensions import Annotated
from typing import Tuple
import numpy as np
import pandas as pd

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin


@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    """Load the digits dataset as numpy arrays."""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))

    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
    )
    return X_train, X_test, y_train, y_test


@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train an sklearn SVC classifier on the digits dataset."""
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    return model


@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
    """Calculate the test set accuracy of an sklearn model."""
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc


In [10]:
from zenml import pipeline

@pipeline
def digits_pipeline():
    """Links all the steps together ina pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

Running Zenmml pipelines

In [11]:
digits_svc_pipeline = digits_pipeline()
# digits_svc_pipeline.run(unlisted=True)

INFO:zenml.pipelines.pipeline_definition:Initiating a new run for the pipeline: `digits_pipeline`.


Initiating a new run for the pipeline: digits_pipeline.


DEBUG:zenml.integrations.integration:Requirement 'pygithub' for integration 'github' is not installed or installed with the wrong version.
DEBUG:zenml.integrations.registry:Integration `github` could not be activated.
DEBUG:zenml.integrations.integration:Integration 'kaniko' is installed correctly with requirements [].
DEBUG:zenml.integrations.registry:Activating integration `kaniko`...
DEBUG:zenml.integrations.registry:Integration `kaniko` is activated.
DEBUG:zenml.integrations.integration:Integration 'jax' is installed correctly with requirements ['jax', 'numpy'].
DEBUG:zenml.integrations.registry:Activating integration `jax`...
DEBUG:zenml.integrations.registry:Integration `jax` is activated.
DEBUG:zenml.integrations.integration:Requirement 'great-expectations>=0.17.15,<1.0' for integration 'great_expectations' is not installed or installed with the wrong version.
DEBUG:zenml.integrations.registry:Integration `great_expectations` could not be activated.
DEBUG:zenml.integrations.inte

Using user: default


INFO:zenml.pipelines.pipeline_definition:Using stack: `default`


...

DEBUG:zenml.zen_stores.sql_zen_store:[N/A] SQL STATS -  'SqlZenStore.create_run_metadata' COMPLETED in 46.45ms   [ active_connections: 1 idle_connections: 1 overflow_connections: -3 ]
DEBUG:zenml.zen_stores.sql_zen_store:[N/A] SQL STATS -  'SqlZenStore.create_artifact_version' COMPLETED in 168.08ms   [ active_connections: 1 idle_connections: 1 overflow_connections: -3 ]
DEBUG:zenml.zen_stores.sql_zen_store:[N/A] SQL STATS -  'SqlZenStore.update_run_step' STARTED  [ active_connections: 0 idle_connections: 2 overflow_connections: -3 ]
DEBUG:zenml.zen_stores.sql_zen_store:[N/A] SQL STATS -  'SqlZenStore.get_server_settings' STARTED  [ active_connections: 1 idle_connections: 1 overflow_connections: -3 ]
DEBUG:zenml.zen_stores.sql_zen_store:[N/A] SQL STATS -  'SqlZenStore.get_server_settings' COMPLETED in 0.99ms   [ active_connections: 2 idle_connections: 0 overflow_connections: -3 ]
DEBUG:zenml.zen_stores.sql_zen_store:[N/A] SQL STATS -  'SqlZenStore.get_user' STARTED  [ active_connections

Step evaluator has finished in 1.098s.


INFO:zenml.orchestrators.local.local_orchestrator:Pipeline run has finished in `3.144s`.


Pipeline run has finished in 3.144s.


DEBUG:zenml.zen_stores.sql_zen_store:[N/A] SQL STATS -  'SqlZenStore.update_run' STARTED  [ active_connections: 0 idle_connections: 2 overflow_connections: -3 ]
DEBUG:zenml.zen_stores.sql_zen_store:[N/A] SQL STATS -  'SqlZenStore.update_run' COMPLETED in 22.92ms   [ active_connections: 1 idle_connections: 1 overflow_connections: -3 ]


In [ ]:
from zenml.environment import Environment
import os

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this URL instead: {public_url}\x1b[0m")

        os.system(f"zenml up --blocking --port {port}")
    else:
        os.system(f"zenml up --port {port}")

start_zenml_dashboard()


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-d5e42ed9-f24b-4f16-978c-d989c777f583


Opening tunnel named: http-8237-d5e42ed9-f24b-4f16-978c-d989c777f583


INFO:pyngrok.process.ngrok:t=2025-12-09T03:12:55+0000 lvl=info msg="no configuration paths supplied"


t=2025-12-09T03:12:55+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2025-12-09T03:12:55+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2025-12-09T03:17:44+0000 lvl=info msg="join connections" obj=join id=e9442efaa5ac l=127.0.0.1:8237 r=[2401:4900:b14e:7347:20a5:8c83:aaaa:a3d9]:53429


INFO:pyngrok.process.ngrok:t=2025-12-09T03:17:44+0000 lvl=info msg="join connections" obj=join id=6ca4cdc6f957 l=127.0.0.1:8237 r=[2401:4900:b14e:7347:20a5:8c83:aaaa:a3d9]:53429


t=2025-12-09T03:17:44+0000 lvl=info msg="join connections" obj=join id=6ca4cdc6f957 l=127.0.0.1:8237 r=[2401:4900:b14e:7347:20a5:8c83:aaaa:a3d9]:53429


INFO:pyngrok.process.ngrok:t=2025-12-09T03:17:44+0000 lvl=info msg="join connections" obj=join id=e8058708f678 l=127.0.0.1:8237 r=[2401:4900:b14e:7347:20a5:8c83:aaaa:a3d9]:53429


t=2025-12-09T03:17:44+0000 lvl=info msg="join connections" obj=join id=e8058708f678 l=127.0.0.1:8237 r=[2401:4900:b14e:7347:20a5:8c83:aaaa:a3d9]:53429
